##**[DACON] 신용카드 고객 세그먼트 분류 AI 경진대회**

### **주제 및 데이터 설명**

**대회 설명**
- 주제: 신용카드 고객 세그먼트 분류 AI 알고리즘 개
- 설명: 일정 기간 동안의 고객 신용정보, 승인 매출, 청구, 잔액, 채널, 마케팅, 성과 등 다양한 데이터를 바탕으로 시용카드 고객 세그먼트(Segment) 분류 AI 알고리즘 개발

**데이터**

train.csv
- 고객별 금융활동 데이터
- 각 정보 내 parquet파일로 구성 (폴더 형태로 구성)
  - 1. 회원정보 (77개 칼럼) + Segment 포함
  - 2. 신용정보 (42개 칼럼)
  - 3. 승인매출정보 (406개 칼럼)
  - 4. 청구입금정보 (46개 칼럼)
  - 5. 잔액정보 (82개 칼럼)
  - 6. 채널정보 (105개 칼럼)
  - 7. 마케팅정보 (64개 칼럼)
  - 8. 성과정보 (49개 칼럼)
- Target: 고객 세그먼트(A~E) [예측 목표]

test.csv
- 고객별 금융활동 데이터
- 각 정보 내 parquet파일로 구성 (Train set과 동일)

데이터 명세.xlsx
- 고객 금융활동 데이터에 대한 명세

### **코드 흐름**

**EDA**

1. Segment 분포 파악 - barplot 이용

  -> E등급이 압도적으로 많은 것을 확인 가능했고, 이후 D등급, C 등급, A등급, B등급 순으로 분포 확인 (A, B 등급의 데이터는 거의 없는 것을 확인 가능)
2. data type 파악
- int64: 747
- float64: 61
- object: 50
3. Outlier (이상치)
- 이상치가 있는 칼럼은 상당히 많은 편으로 나타남
- object로 전환시켜주어야 하거나 피처엔지니어링이 필요한 변수들(int, float형이 아닌 항목)이 다수 보임
4. Missing (결측치)
- 가입통신회사코드, 직장시도명, _1순위신용체크구분, _2순위신용체크구분, 최종유효년월_신용_이용가능, 최종유효년월_신용_이용, 최종카드발급일자 등 다양한 부분에서 결측치 발생 확인
- object type의 경우 칼럼의 약 40% 가량 결측치 존재 확인

**Feature Engineering & preprocessing**
- Model을 2가지로 구분: BASE Model(C,D,E 클래스), VIP Model(A,B 클래스)
  - 예측 성능인 Macro F1 특성 상 A와 B 클래스 예측 정확도가 전체 성적에 큰 영향을 주는 것으로 판단 -> A,B 클래스만 VIP 클래스로 분류하여 따로 예측
- Train 데이터 기준
  - 모든 값이 동일한 칼럼 제거
  - 중복 칼럼 제거

  -> 최종 746개의 칼럼을 가지고 진행
- Test 데이터 : 최종 745개의 칼럼으로 진행 (Segment 칼럼 제외)

**전반적인 모델 흐름**

CatBoost + 2-way optimization 전략 사용

1. BASE Model 흐름
- Train에서 Segment A,B 제거
- Optuna 활용 하이퍼파라미터 튜닝 (iterations = 1500, depth=8, class weight=[2,1,1]
- Catboost를 활용한 KFold 확률 평균 기반 Segment 예측

2. VIP Model 흐름
- Segment A 필터 후 동일한 값들을 찾아 해당 유형을 띄는 ID를 필터링하여 train-test 재구성
- Catboost를 사용한 A값 예측(iterations=2000, depth=6, class weight=[20, 50, 2, 1, 1]
- Segment B 필터 후 동일한 값들을 찾아 해당 유형을 띄는 ID들을 필터링하여 train-test 재구성
- Catboost를 사용하여 B값을 예측 (iterations=1000, depth=8, class weight= [10, 10, 1, 1, 1]
- CDE를 예측한 BASE Model에 A값과 B값을 덮어씌워 Submission을 완성

**성능향상을 위한 코드의 차별점**
1. GPU의 사용
- 해당 조에서 성능 향상에 가장 큰 영향을 준 요소로 판단
- 코랩 기준 A100 GPU와 L4 GPU를 활용 => 점수가 0.68 후반대로 상승할 수 있던 전략

2. VIP Model의 분리
- Macro F1의 특성상 높은 클래스인 A와 B의 예측 정확도가 전체 성적에 큰 영향을 준다고 판단
- 해당 클래스들 만들 따로 학습하는 모델을 형성 (A와 B도 동시에 학습한 것이 아닌 각각 따로 학습하도록 코드를 구성) => 점수가 0.7로 오르게 됨

3. VIP Model의 보완
- B의 경우 1개 외에는 확장이 어렵다는 결론에 도달하게 됨 -> A 클래스 예측에 집중
- A 클래스 예측에서 SMOTE, Soft/Hard Voting, Ensemble 등 다양한 클래스를 실험 -> CatBoost가 예측한 A클래스에 포함되거나 유의미한 점수차이를 확인할 수 없었음

   => 최종 CatBoost 단일모델로 확정 ( BASE Model과 VIP Model모두 CatBoost 단일모델로 구성)

4. C와 D의 혼동 분류
- C 클래스의 예측 성능을 높이기 위한 class weight 조정 시도 -> 점수의 유의미한 상승 확인 가능
- AB Segment 제거와 overfitting을 고려한 Optuna의 학습 또한 긍정적 영향 예측

**새롭게 알게된 점**
1. 코랩 내 GPU 변경의 영향
- 모델 학습 속도 및 추론 속도의 향상 : 딥러닝 모델 학습이 빨라지고 예측값 계산 또한 빠르게 수행가능
- 큰 모델 처리 가능: 고성능 GPU를 선택할 시 더욱 큰 배치 사이즈, 복잡한 신경망 모델을 처리할 수 있어 과적합 방지 실험에도 유의미한 영향을 미침
- 병렬 처리의 효율 증가: 고성능 GPU의 경우 병렬 연산을 더욱 빠르게 처리하게 되어 다수의 샘플에 대한 예측값을 동시에 계산 가능
- 코랩 내에서 사용할 수 있는 주요 GPU
  - NVIDIA A100(최대 40GB): 매우 빠르고 대형 모델 학습 가능(유료버전)
  - NVIDIA V100(최대 16GB): 고성능, 빠른 학습 및 추론 가능
  - NVIDIA T4(16GB): 중간성능, 무료로 사용가능 (대다수 이게 할당됨)
  - NVIDIA P100(16GB): T4보다 약간 빠름(일부 pro 사용자에게 할당)
- 코랩의 현재 GPU 정보를 확인하는 방법은 아래 코드를 이용
```
!nvidia-smi
```
- 본 모델에서 사용한 A100의 경우 학습과 추론에 모두 강력한 GPU이고 L4의 경우 추론용으로 최적화된 효율적인 GPU로 알려짐

2. 각 Segment별 예측성능을 높이기 위한 노력
- 전반적인 모델의 예측성능을 높이기 위해 하이퍼 파라미터 튜닝을 진행한 것이 아닌, 각 Segment 별 예측성능을 높이고자 A,B 클래스 분리 Model 형성 + C 클래스 classweight 조정을 진행
  -> 각 클래스에 대한 높은 예측성능을 가진 모델을 형성함
- 특정 클래스 예측 성능을 개선하기 위해서는 해당 클래스의 데이터를 더욱 자세히 들여다 보는 작업이 필요함
  - 클래스 별 데이터 분포 확인: 각 클래스의 샘플이 균형적인지, 소수 클래스가 존재하는지, 어떤 클래스의 예측을 많이 틀리는지 확인 필요
  - 클래스별 특성(Feature) 탐색: 특정 클래스가 다른 클래스와 어떤 feature에서 뚜렷하게 구분되는지 분포, 상관관계 이상치 등의 시각화로 파악하기
  - 모델이 많이 틀리는 클래스 조사: confusion matrix로 확인하기 (오류 패턴 파악)
  - 클래스별 예측 확률 분포 확인: 어떤 클래스가 예측 확률이 낮고 어떤 클래스가 뚜렷한 확률을 가지는지 확인하여 threshold 조정하기
- 얻을 수 있는 효과
  - 클래스별 특성을 반영한 feature engineering 가능
  - 데이터 편향이나 노이즈 문제를 사전에 파악 가능
  - 모델 성능 향상 전략을 정교하게 수립 가능
- 각 Class 별 예측 성능을 높이는 전략
  - 클래스별 가중치 적용(classweight): 모델을 학습하는 과정에서 자주 틀리는 클래스나 중요도가 높은 클래스에 loss penalty를 크게 부여하여 예측 성능을 향상할 수 있도록 함
  - 클래스별 샘플링 전략 조정
    - 오버샘플링: 소수 클래스 데이터를 SMOTE 등을 이용하여 인위적으로 늘림
    - 언더샘플링: 다수 클래스 데이터를 줄여 데이터간 균형을 유지하도록 함
    - 클래스별 샘플링 비율을 다르게 적용하여 균형을 맞춤
  - 클래스별 특화 feature engineering: 특정 클래스가 어떤 클래스의 변화에 민감하면 그에 맞는 새로운 변수를 생성하거나 변환

3. 예측에 사용한 CatBoost 모델

    CatBoost란?

    CatBoost: 범주형 변수처리에 특화되어 있으며 성능이 뛰어난 Gradient Boosting 모델
  - 범주형 변수가 많은 경우 따로 Label Encoding이나 One-hot encoding을 수행하지 않아도 문자형 변수를 그대로 넣으면 알아서 처리해줌
  - 과적합 억제에 강함: Ordering boosting 기법을 사용하여 데이터 누수 방지
  - 빠른 학습 속도: CPU, GPU를 모두 지원하며 내부 최적화가 잘 되어있어 속도가 빠름
  - 결측값의 자동처리: 결측값을 확인하기 어려운 복잡한 데이터에서 NaN 값을 따로 전처리 하지 않아도 결측값을 자동적으로 처리해줌

  => 즉 CatBoost는 XGBoost와 LightGBM에 비해 과적합 제어와 성능이 뛰어나다는 점이 금융 데이터와 같은 복잡한 데이터를 다루는데 유리한 모델이 될 수 있음

4. Optuna를 이용한 하이퍼 파라미터 튜닝

    Optuna란?

    Optuna: 모델의 하이퍼파라미터를 효육적으로 찾아주는 최적화 라이브러리
    - 하이퍼 파라미터 최적화 도구로 모델 성능을 최대화하고 시행착오를 줄이기 위해 사용하는 라이브러리
    - 베이지안 최적화 방법 + 가지치기(Pruning) 방법을 사용하며 이전 시도들을 학습하여 좋은 방향으로만 탐색하고, 성능이 낮을 것 같은 조합은 중간에 실험을 끊음
    - 같은 조건에 GridSearch보다 훨씬 적은 시도로 높은 성능을 달성하고, 병렬화도 간단히 가능하기 때문에 CPU나 GPU 자원 활용이 효율적임
    - 단순한 objective 함수로 튜닝이 가능하고 CatBoost, LightGBM 등의 기본 연동이 빠름
    - hyperopt와는 유사한 방법일 수 있으나 Hyperopt의 경우 분산환경에서 효율적인 최적화가 가능하고 복잡한 하이퍼 파라미터 공간에서 유리한 반면, Optuna는 빠르고 효율적인 하이퍼파라미터 튜니이 가능하고 조기 종료 기능적인 면에서 더욱 강력하고, 직관적인 사용이 가능하다.

**전반적인 코드 확인 후 느낀점**
- GPU 성능을 바꿔 모델의 전반적인 속도를 향상시키는 방법도 생각해보아야 한다는 것을 깨달음
- 단순히 전체적인 하이퍼파라미터 튜닝만 생각해보았는데, 데이터를 더욱 면밀히 파악해서 데이터에 적합한 각 Class별 튜닝이나 예측 성능을 높이기 위한 전략도 수립해야 전반적인 예측 성능 점수가 올라갈 수 있다는 것을 확인함
- 추가적으로 균형데이터가 아닌 불균형 데이터의 경우 모델을 2가지로 분류하는 전략을 세워 각 모델을 따로 학습하고 이를 합치는 방법도 생각해 볼 수 있었음
- XGBoost, LightGBM, 랜덤포레스트 외에도 CatBoost라는 모델을 이용하여 분류 예측이 가능하다는 것을 확인할 수 있고, 배운 모델보다 더욱 복잡하고 방대한 데이터를 다룰 때 적합한 모델이라는 것을 확인할 수 있었음
- 하이퍼파라미터 튜닝으로 GridSearchCV, RandomizedSearchCV, Hyperopt만을 생각했었는데, Optuna라는 다른 튜닝 방법도 생각해볼 수 있었고, 코드가 간단하다는 점이 굉장히 강력한 튜닝 모델이라는 것을 생각해 볼 수 있었음